# Data Preprocessing
While from imagenet might need little processing. To retrain Resnet-50 with adversarial inputs, they need to be generated. Training dataset has about 1M images, preprocessing, generates adversarial input for both training and testing datasets.
Corresponding adversarial tensorflow records are created for both datasets.

In [1]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import preprocess_input

# Constants
IMG_SIZE = 224
# BATCH_SIZE = 5000
BATCH_SIZE = 200
AUTOTUNE = tf.data.AUTOTUNE
EPOCHS = 5
tf.random.set_seed(5)

physical_gpus = tf.config.list_physical_devices('GPU')
print("Available GPUs:", physical_gpus)

try:
    tf.keras.mixed_precision.set_global_policy('float32')
    tf.config.experimental.set_virtual_device_configuration(
        physical_gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=56320)]  # Limit RAM to 55GB to avoid starving PC
    )
    print("Using GPU with 55GB of memory")
except Exception as e:
    print(e)


Available GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Using GPU with 55GB of memory


In [2]:
# Load ImageNet data

def prepare_input_data(input):
    image = tf.cast(input['image'], tf.float32) # ResNet-50 used this
    image = tf.image.resize(input['image'], (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)
    label = input['label']
    return image, label

# Big dataset for real work
# dataset, info = tfds.load(
#     'imagenet2012',
#     shuffle_files=False,
#     with_info=True,
#     data_dir='../datasets'
# )

# Smaller dataset for testing
dataset, info = tfds.load(
    'imagenette',
    shuffle_files=False,
    with_info=True,
    data_dir='../datasets'
)

# Dataset stats
print(f'Train image count: {info.splits['train'].num_examples}')
print(f'Test image count: {info.splits['validation'].num_examples}')

# Preprocess data
train_dataset = dataset['train'].map(prepare_input_data, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)
test_dataset = dataset['validation'].map(prepare_input_data, num_parallel_calls=AUTOTUNE).batch(BATCH_SIZE).prefetch(AUTOTUNE)

Train image count: 9469
Test image count: 3925


2025-08-05 23:45:16.984447: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Max
2025-08-05 23:45:16.984475: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2025-08-05 23:45:16.984482: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2025-08-05 23:45:16.984501: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-08-05 23:45:16.984525: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Load ResNet50 model
from tensorflow.keras.applications import ResNet50

base_model = ResNet50(
    include_top=True,
    weights='imagenet',
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    pooling=None,
    classes=1000,
    classifier_activation='softmax'
)

In [4]:
# save_processed_dataset('../datasets/adversaries/test_dataset.tfrecord', adversarial_test_dataset)
# %reset_selective -f adversarial_test_dataset test_dataset
# import gc;gc.collect()

In [5]:
def _bytes_feature(value):
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # get value from EagerTensor
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def _create_adversary_with_pgd(model, images, labels, eps, eps_iter, nb_iter):
    """
    This generates adversarial images by iteratively applying a small
    perturbation in the direction of the gradient of the loss, and then
    projecting the result back into the epsilon-ball of the original image.

    Args:
        model (tf.keras.Model): The model to attack.
        images (tf.Tensor): The original, clean input images.
        labels (tf.Tensor): The true labels for the images.
        eps (float): The maximum perturbation (L-infinity norm).
        eps_iter (float): The step size for each attack iteration.
        nb_iter (int): The number of PGD iterations to perform.

    Returns:
        tf.Tensor: The generated adversarial images.
    """
    x_adv = tf.identity(images)
    loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

    for _ in range(nb_iter):
        with tf.GradientTape() as tape:
            tape.watch(x_adv)
            prediction = model(x_adv, training=False)
            loss = loss_object(labels, prediction)

        gradients = tape.gradient(loss, x_adv)
        signed_grad = tf.sign(gradients)
        x_adv = x_adv + eps_iter * signed_grad
        perturbation = tf.clip_by_value(x_adv - images, -eps, eps)
        x_adv = images + perturbation

    return x_adv

def generate_adversarial_dataset(filename,dataset, model, eps, pgd_steps, pgd_step_size):
    """
    Generates adversarial examples and saves them to a TFRecord file
    by serializing the raw float32 tensors.
    """
    options = tf.io.TFRecordOptions(compression_type="GZIP")
    num = 0
    with tf.io.TFRecordWriter(filename, options=options) as writer:
        for i, (images, labels) in enumerate(dataset):
            print(f"Batch {i+1}")
            # Generate the adversarial images (these are already preprocessed)
            adv_images = _create_adversary_with_pgd(
                model=model,
                images=images,
                labels=labels,
                eps=eps,
                eps_iter=pgd_step_size,
                nb_iter=pgd_steps
            )

            # Iterate through the batch to save each image/label pair
            for i in range(len(adv_images)):
                image_tensor = adv_images[i]
                label = labels[i]

                # 1. Cast the tensor to float16 to halve its size
                image_tensor_f16 = tf.cast(image_tensor, tf.float16)

                # 2. Serialize the smaller tensor
                image_bytes = tf.io.serialize_tensor(image_tensor_f16)
                # 2. Create the feature and write to the TFRecord file
                feature = {
                    'image': _bytes_feature(image_bytes), # Save the raw serialized tensor
                    'label': _int64_feature(label.numpy())
                }
                num += 1
                serialized_example = tf.train.Example(features=tf.train.Features(feature=feature)).SerializeToString()
                writer.write(serialized_example)

    print(f"Processed and saved: {num} images")


In [6]:
## Create adversarial dataset
EPSILON = 0.03
PGD_STEPS = 2
PGD_STEP_SIZE = 0.007
adversarial_test_file = '../datasets/adversaries/small_test_dataset.tfrec'
# generate_adversarial_dataset(
#     filename=adversarial_test_file,
#     dataset=test_dataset,
#     model=base_model,
#     eps=EPSILON,
#     pgd_steps=PGD_STEPS,
#     pgd_step_size=PGD_STEP_SIZE
# )
adversarial_train_file = '../datasets/adversaries/small_train_dataset.tfrec'
# generate_adversarial_dataset(
#     filename=adversarial_train_file,
#     dataset=train_dataset,
#     model=base_model,
#     eps=EPSILON,
#     pgd_steps=PGD_STEPS,
#     pgd_step_size=PGD_STEP_SIZE
# )

In [7]:
## Load data from file
def _parse_function(proto):
    """
    Parses a single example proto by deserializing the float16 tensor
    and casting it back to float32.
    """
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    parsed_features = tf.io.parse_single_example(proto, feature_description)

    # 1. Deserialize the byte string back into a float16 tensor
    image_f16 = tf.io.parse_tensor(parsed_features['image'], out_type=tf.float16)
    label = parsed_features['label']

    # 2. Cast the image back to float32 for the model
    image_f32 = tf.cast(image_f16, tf.float32)

    # 3. Set the shape on the final float32 tensor
    image_f32.set_shape([IMG_SIZE, IMG_SIZE, 3])

    return image_f32, label

# Load the TFRecord file back into a dataset
loaded_test_dataset = tf.data.TFRecordDataset(adversarial_test_file, compression_type='GZIP')

# Map the parsing function across the dataset
parsed_test_dataset = loaded_test_dataset.map(_parse_function).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [8]:
# Compile model
base_model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [9]:
print("Baseline accuracy")
# results = base_model.evaluate(test_dataset, verbose=0, batch_size=10, steps=2)

train_loss, train_accuracy = base_model.evaluate(train_dataset, verbose=0)
test_loss, test_accuracy = base_model.evaluate(test_dataset, verbose=0)
noisy_loss, noisy_accuracy = base_model.evaluate(parsed_test_dataset, verbose=0)

# The following code prints the formatted report
print("\n" + "="*50)
print("       Comprehensive Model Evaluation Results")
print("="*50 + "\n")

print("## Accuracy & Error Rate")
print("**Calculated on the clean, unseen test dataset.**\n")
print(f"* **Test Accuracy**: The percentage of correct predictions.\n    * `{test_accuracy * 100:.2f}%`")
print(f"* **Test Error Rate**: The percentage of incorrect predictions (`1 - Accuracy`).\n    * `{(1 - test_accuracy) * 100:.2f}%`")
print("\n" + "---")

print("\n## Generalisation 🧠")
print("**Measures the model's ability to perform on unseen data by comparing training and test performance.**\n")
print(f"* **Training Accuracy**: Performance on data the model was trained on.\n    * `{train_accuracy * 100:.2f}%`")
print(f"* **Test Accuracy**: Performance on new, unseen data.\n    * `{test_accuracy * 100:.2f}%`")
print(f"* **Generalisation Gap**: The difference between training and test accuracy.\n    * `{(train_accuracy - test_accuracy) * 100:.2f}%`")
print("> *A small gap is ideal. A large gap suggests the model has **overfit** to the training data.*")
print("\n" + "---")

print("\n## Robustness 🛡️")
print("**Measures how well performance holds up when the input data is perturbed.**\n")
print(f"* **Accuracy (Clean Data)**: Baseline performance on the original test set.\n    * `{test_accuracy * 100:.2f}%`")
print(f"* **Accuracy (Noisy Data)**: Performance on the perturbed test set.\n    * `{noisy_accuracy * 100:.2f}%`")
print(f"* **Performance Drop**: The reduction in accuracy due to the noisy data.\n    * `{(test_accuracy - noisy_accuracy) * 100:.2f}%`")
print("> *A smaller drop indicates a more robust model.*")

Baseline accuracy


2025-08-05 23:45:19.295026: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.



       Comprehensive Model Evaluation Results

## Accuracy & Error Rate
**Calculated on the clean, unseen test dataset.**

* **Test Accuracy**: The percentage of correct predictions.
    * `9.04%`
* **Test Error Rate**: The percentage of incorrect predictions (`1 - Accuracy`).
    * `90.96%`

---

## Generalisation 🧠
**Measures the model's ability to perform on unseen data by comparing training and test performance.**

* **Training Accuracy**: Performance on data the model was trained on.
    * `9.39%`
* **Test Accuracy**: Performance on new, unseen data.
    * `9.04%`
* **Generalisation Gap**: The difference between training and test accuracy.
    * `0.34%`
> *A small gap is ideal. A large gap suggests the model has **overfit** to the training data.*

---

## Robustness 🛡️
**Measures how well performance holds up when the input data is perturbed.**

* **Accuracy (Clean Data)**: Baseline performance on the original test set.
    * `9.04%`
* **Accuracy (Noisy Data)**: Performance on th

2025-08-05 23:52:14.284432: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
2025-08-05 23:52:14.284530: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 8286550425692185148
2025-08-05 23:52:14.284545: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
	 [[{{node IteratorGetNext}}]]
	 [[StatefulPartitionedCall/BitwiseAnd/_12]]
2025-08-05 23:52:14.284586: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 10412056754719796095
2025-08-05 23:52:14.284600: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 11045578175169975825
2025-08-05 23:52:14.284626: I tensorflow/core/framework/local_rendezvous.cc:422] Local rendezvous recv item cancelled. Key hash: 659276063

In [10]:
# save_processed_dataset('../datasets/adversaries/train_dataset.tfrecord', adversarial_train_dataset)
# %reset_selective -f adversarial_train_dataset train_dataset
# import gc;gc.collect()

In [ ]:
## Adversarial training
# Load the TFRecord file back into a dataset
loaded_train_dataset = tf.data.TFRecordDataset(adversarial_train_file, compression_type='GZIP')

# Map the parsing function across the dataset
parsed_train_dataset = loaded_train_dataset.map(_parse_function).batch(BATCH_SIZE).repeat().prefetch(tf.data.AUTOTUNE)

history = base_model.fit(
    parsed_train_dataset,
    epochs=10, # Use a suitable number of epochs for your task
    validation_data=parsed_test_dataset,
    verbose=0
)


In [ ]:
print("Robustness model accuracy")
# base_model.compile(optimizer='adam',
#                 loss='sparse_categorical_crossentropy',
#                 metrics=['accuracy'])
# results = base_model.evaluate(test_dataset, verbose=0, batch_size=10, steps=2)

train_loss, train_accuracy = base_model.evaluate(train_dataset, verbose=0)
test_loss, test_accuracy = base_model.evaluate(test_dataset, verbose=0)
noisy_loss, noisy_accuracy = base_model.evaluate(parsed_test_dataset, verbose=0)

# The following code prints the formatted report
print("\n" + "="*50)
print("       Comprehensive Model Evaluation Results")
print("="*50 + "\n")

print("## Accuracy & Error Rate")
print(f"* **Test Accuracy**:   `{test_accuracy * 100:.2f}%`")
print(f"* **Test Error Rate**: (`1 - Accuracy`).\n    * `{(1 - test_accuracy) * 100:.2f}%`")
print("\n" + "---")

print("\n## Generalisation 🧠")
print("**Measures the model's ability to perform on unseen data by comparing training and test performance.**\n")
print(f"* **Training Accuracy**: Performance on data the model was trained on.\n    * `{train_accuracy * 100:.2f}%`")
print(f"* **Test Accuracy**: Performance on new, unseen data.\n    * `{test_accuracy * 100:.2f}%`")
print(f"* **Generalisation Gap**: The difference between training and test accuracy.\n    * `{(train_accuracy - test_accuracy) * 100:.2f}%`")
print("> *A small gap is ideal. A large gap suggests the model has **overfit** to the training data.*")
print("\n" + "---")

print("\n## Robustness 🛡️")
print("**Measures how well performance holds up when the input data is perturbed.**\n")
print(f"* **Accuracy (Clean Data)**: Baseline performance on the original test set.\n    * `{test_accuracy * 100:.2f}%`")
print(f"* **Accuracy (Noisy Data)**: Performance on the perturbed test set.\n    * `{noisy_accuracy * 100:.2f}%`")
print(f"* **Performance Drop**: The reduction in accuracy due to the noisy data.\n    * `{(test_accuracy - noisy_accuracy) * 100:.2f}%`")
print("> *A smaller drop indicates a more robust model.*")